# 02 - Tiền xử lý dữ liệu HR Analytics: Job Change

Trong notebook này, chúng ta sẽ:

- Đọc dữ liệu gốc bằng NumPy.
- Kiểm tra tính hợp lệ của dữ liệu (missing, giá trị lạ).
- Xử lý missing values.
- Phát hiện và xử lý ngoại lai (outlier) bằng IQR.
- Áp dụng các kỹ thuật chuẩn hoá: Min-Max, Z-score, log transform, decimal scaling.
- Thực hiện một số feature engineering đơn giản.
- Tính toán thống kê mô tả và minh hoạ một ví dụ kiểm định giả thiết thống kê (H0, H1).
- Lưu dữ liệu đã xử lý ra `data/processed/`.


In [ ]:
import os
import sys
import numpy as np

BASE_DIR = os.path.abspath("..")
sys.path.append(BASE_DIR)
DATA_PATH = os.path.join(BASE_DIR, "data", "raw", "aug_train.csv")  # đổi tên nếu cần
PROCESSED_DIR = os.path.join(BASE_DIR, "data", "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

from src.data_processing import (
    load_dataset,
    build_feature_matrix,
    min_max_scale,
    standardize_zscore,
    log_transform,
    decimal_scaling,
    describe_numeric,
    iqr_outlier_mask,
)

print("BASE_DIR:", BASE_DIR)



PROCESSED_PATH = os.path.join(PROCESSED_DIR, "hr_data_processed.csv")

print("DATA_PATH:", DATA_PATH)
print("PROCESSED_PATH:", PROCESSED_PATH)
print("File DATA tồn tại?", os.path.exists(DATA_PATH))


BASE_DIR: d:\HOCTAP\LTKHDL\Homework2
DATA_PATH: d:\HOCTAP\LTKHDL\Homework2\data\raw\aug_train.csv
PROCESSED_PATH: d:\HOCTAP\LTKHDL\Homework2\data\processed\hr_data_processed.csv
File DATA tồn tại? True


In [2]:
header, data = load_dataset(DATA_PATH)

print("Số dòng:", data.shape[0])
print("Số cột:", data.shape[1])
print("Tên cột:", header)


Số dòng: 19158
Số cột: 14
Tên cột: ['enrollee_id', 'city', 'city_development_index', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job', 'training_hours', 'target']


In [3]:
# Đếm missing từng cột
print("=== Số lượng missing theo cột ===")
for i, name in enumerate(header):
    col = data[:, i].astype(str)
    missing = np.sum((col == "") | (col == "NA") | (col == "NaN") | (col == "null") | (col == "NULL") | (col == "?"))
    print(f"{name}: {missing}")

# Ví dụ: kiểm tra domain cho một vài cột
if "target" in header:
    t_idx = header.index("target")
    target_vals, target_counts = np.unique(data[:, t_idx], return_counts=True)
    print("\nGiá trị target:", target_vals, target_counts)

if "experience" in header:
    e_idx = header.index("experience")
    exp_vals, exp_counts = np.unique(data[:, e_idx], return_counts=True)
    print("\nMột vài giá trị experience:", exp_vals[:20])


=== Số lượng missing theo cột ===
enrollee_id: 0
city: 0
city_development_index: 0
gender: 4508
relevent_experience: 0
enrolled_university: 386
education_level: 460
major_discipline: 2813
experience: 65
company_size: 5938
company_type: 6140
last_new_job: 423
training_hours: 0
target: 0

Giá trị target: ['0.0' '1.0'] [14381  4777]

Một vài giá trị experience: ['' '1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '3' '4'
 '5' '6' '7' '8']


## Chọn các cột numeric, categorical và label

- Numeric:
  - `city_development_index`
  - `training_hours`
- Categorical:
  - `city`, `gender`, `relevent_experience`, `enrolled_university`,
    `education_level`, `major_discipline`, `experience`,
    `company_size`, `company_type`, `last_new_job`
- Label:
  - `target`


In [4]:
numeric_cols = ["city_development_index", "training_hours"]

categorical_cols = [
    "city",
    "gender",
    "relevent_experience",
    "enrolled_university",
    "education_level",
    "major_discipline",
    "experience",
    "company_size",
    "company_type",
    "last_new_job",
]

label_col = "target"

print("Numeric cols:", numeric_cols)
print("Categorical cols:", categorical_cols)
print("Label col:", label_col)


Numeric cols: ['city_development_index', 'training_hours']
Categorical cols: ['city', 'gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job']
Label col: target


In [5]:
X, y, meta = build_feature_matrix(
    header=header,
    data=data,
    numeric_cols=numeric_cols,
    categorical_cols=categorical_cols,
    label_col=label_col,
)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Label mapping:", meta["label_mapping"])
print("Số feature numeric:", len(numeric_cols))
print("Tổng số feature:", X.shape[1])


X shape: (19158, 186)
y shape: (19158,)
Label mapping: {np.str_('0.0'): 0, np.str_('1.0'): 1}
Số feature numeric: 2
Tổng số feature: 186


In [6]:
# Numeric nằm ở các cột đầu tiên (theo build_feature_matrix)
n_num = len(numeric_cols)
X_num = X[:, :n_num]

stats = describe_numeric(X_num)
print("=== Thống kê mô tả cho numeric features ===")
print("Thứ tự:", numeric_cols)
for k, v in stats.items():
    print(k, ":", v)


=== Thống kê mô tả cho numeric features ===
Thứ tự: ['city_development_index', 'training_hours']
mean : [ 0.828848   65.36689634]
std : [ 0.12335854 60.05689445]
min : [0.448 1.   ]
25% : [ 0.74 23.  ]
50% : [ 0.903 47.   ]
75% : [ 0.92 88.  ]
max : [  0.949 336.   ]


In [7]:
for j, col_name in enumerate(numeric_cols):
    col = X_num[:, j]
    out_mask, bounds = iqr_outlier_mask(col, factor=1.5)
    num_out = np.sum(out_mask)
    print(f"Cột {col_name}: {num_out} outliers, bounds = {bounds}")


Cột city_development_index: 17 outliers, bounds = (np.float64(0.4699999999999999), np.float64(1.1900000000000002))
Cột training_hours: 984 outliers, bounds = (np.float64(-74.5), np.float64(185.5))


## Chuẩn hoá & Điều chuẩn dữ liệu

Ở đây ta sẽ thử:

- **Min-Max Scaling**: đưa từng feature về [0, 1].
- **Z-score Standardization**: đưa về mean ~ 0, std ~ 1 (hữu ích cho thuật toán dựa trên gradient).
- **Log transform** cho `training_hours` (nếu phân phối lệch).
- **Decimal scaling** nếu cần cho một vài biến nhiều chữ số.


In [8]:
X_minmax, mins, maxs = min_max_scale(X)
print("Sau Min-Max:", X_minmax.shape)

X_zscore, means, stds = standardize_zscore(X)
print("Sau Z-score:", X_zscore.shape)


Sau Min-Max: (19158, 186)
Sau Z-score: (19158, 186)


In [9]:
# training_hours là numeric_cols[1]
training_idx_in_X = numeric_cols.index("training_hours")
col_train = X[:, training_idx_in_X]

col_train_log, shift = log_transform(col_train)
print("Dịch chuyển (shift) để log training_hours:", shift)
print("Trước log: min =", np.min(col_train), ", sau log: min =", np.min(col_train_log))


Dịch chuyển (shift) để log training_hours: 0.0
Trước log: min = 1.0 , sau log: min = 0.0


In [10]:
X_dec_scaled, ds = decimal_scaling(X)
print("Decimal scaling xong, ví dụ d (first 10):", ds[:10] if isinstance(ds, np.ndarray) else ds)


Decimal scaling xong, ví dụ d (first 10): [0 3 0 0 0 0 0 0 0 0]


## Feature Engineering đơn giản

Ví dụ:

- Tạo feature `city_dev_x_training` = `city_development_index * training_hours`
  - Ý tưởng: ứng viên ở thành phố phát triển cao và dành nhiều thời gian training
    có thể có hành vi khác (muốn đổi việc nhiều hơn).


In [11]:
X_fe = X_minmax.copy()  # dùng bản đã Min-Max để dễ kết hợp

idx_city_dev = numeric_cols.index("city_development_index")
idx_training = numeric_cols.index("training_hours")

city_dev = X_minmax[:, idx_city_dev]
training = X_minmax[:, idx_training]

city_dev_x_training = city_dev * training
city_dev_x_training = city_dev_x_training.reshape(-1, 1)

X_fe = np.hstack([X_fe, city_dev_x_training])

print("X_fe shape (sau khi thêm feature):", X_fe.shape)


X_fe shape (sau khi thêm feature): (19158, 187)


In [12]:
# Lấy training_hours từ X (trước scale cho dễ hiểu)
idx_training = numeric_cols.index("training_hours")
training_raw = X[:, idx_training]  # đã được fill missing

# target là 0/1 trong y
group0 = training_raw[y == 0]
group1 = training_raw[y == 1]

mean0 = np.mean(group0)
mean1 = np.mean(group1)
std0 = np.std(group0, ddof=1)
std1 = np.std(group1, ddof=1)
n0 = group0.size
n1 = group1.size

print("Mean training_hours, target=0:", mean0)
print("Mean training_hours, target=1:", mean1)

# t-test 2 sample độc lập (giả sử var khác nhau)
se = np.sqrt(std0**2 / n0 + std1**2 / n1)
t_stat = (mean0 - mean1) / se

print("t-statistic:", t_stat)

# Không tính p-value chi tiết (cần hàm CDF),
# nhưng bạn có thể so sánh |t| với ngưỡng (ví dụ ~1.96 cho alpha=0.05)


Mean training_hours, target=0: 66.11376121271121
Mean training_hours, target=1: 63.11848440443793
t-statistic: 3.0813601884304807


## Lưu dữ liệu đã xử lý

Chọn một bản X để dùng cho modeling, ví dụ:

- `X_fe` : Min-Max + thêm feature `city_dev_x_training`.

Ta sẽ lưu:
- Cột đầu: `y`
- Các cột còn lại: feature đã xử lý.


In [ ]:
y_col = y.reshape(-1, 1).astype(float)
out = np.hstack([y_col, X_fe])

np.savetxt(
    PROCESSED_PATH,
    out,
    delimiter=",",
    fmt="%.6f",
)

print("Đã lưu dữ liệu đã xử lý tại:", PROCESSED_PATH)


Đã lưu dữ liệu đã xử lý tại: d:\HOCTAP\LTKHDL\Homework2\data\processed\hr_data_processed.csv
1 dòng mẫu (y, x1, x2, ...):
[1.         0.94211577 0.10447761 0.         0.         0.
 0.         0.         1.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.     